In [1]:
%matplotlib inline

import gym
import matplotlib
import numpy as np
import sys

from collections import defaultdict
if "../" not in sys.path:
    sys.path.append("../") 
from lib.envs.blackjack import BlackjackEnv
from lib import plotting

matplotlib.style.use('ggplot')

In [2]:
env = BlackjackEnv()

In [3]:
def create_random_policy(nA):
    """
    Creates a random policy function.
    
    Args:
        nA: Number of actions in the environment.
    
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities
    """
    A = np.ones(nA, dtype=float) / nA
    def policy_fn(observation):
        return A
    return policy_fn

In [4]:
def create_greedy_policy(Q):
    """
    Creates a greedy policy based on Q values.
    
    Args:
        Q: A dictionary that maps from state -> action values
        
    Returns:
        A function that takes an observation as input and returns a vector
        of action probabilities.
    """
    A = np.divide(np.ones(nA, dtype=np.float32)*epsilon, nA)
    def policy_fn(observation):
        opt_action = randargmax(Q[observation])
        A[opt_action] += (1.0 - epsilon)
        return A
    return policy_fn

In [ ]:
def mc_control_importance_sampling(env, num_episodes, behavior_policy, discount_factor=1.0):
    """
    Monte Carlo Control Off-Policy Control using Weighted Importance Sampling.
    Finds an optimal greedy policy.
    
    Args:
        env: OpenAI gym environment.
        num_episodes: Number of episodes to sample.
        behavior_policy: The behavior to follow while generating episodes.
            A function that given an observation returns a vector of probabilities for each action.
        discount_factor: Gamma discount factor.
    
    Returns:
        A tuple (Q, policy).
        Q is a dictionary mapping state -> action values.
        policy is a function that takes an observation as an argument and returns
        action probabilities. This is the optimal greedy policy.
    """
    
    def generate_episode(env, policy):
        episode = []
        state = env.reset()
        while True:
            probs = policy(state)
            action = np.random.choice(np.arange(len(probs)), p=probs)
            next_state, reward, done, _ = env.step(action)
            episode.append(state)
            episode.append(action)
            episode.append(reward)
            if done:
                break
            state = next_state
        return episode
    
    # Cumulative sum of returns and weights
    returns_sum = defaultdict(float)
    weights_sum = defaultdict(float)
    
    # A nested dictionary that maps state -> action values (numpy array of size nA)
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # Our greedy policy we want to learn
    # With ties broken consistently
    target_policy = create_greedy_policy(Q)
    
    # Loop for each episode
    for e in range(1, num_episodes+1):
        if e % 1000 == 0:
            print("\rEpisode {}/{}.".format(e, num_episodes), end="")
            sys.stdout.flush()

        # generate an episode following policy
        episode = generate_episode(env, policy)
        state_action = set() # store unique state-action pairs
        # Loop for each step of episode
        for state_idx in range(0, len(episode), 3):
            state = episode[state_idx]
            action = episode[state_idx+1]
            sap = (state, action)
            # consider only first occurence of each state-action pair
            if sap not in state_action:
                state_action.add(sap)
                returns_count[sap] += 1
                # increment sum of returns
                returns_sum[sap] += sum([ discount_factor**i * episode[reward_idx] for i, reward_idx in enumerate(range(state_idx+2, len(episode), 3)) ])
                # update state-action value by averaging the returns of that state-action pair over all episodes
                Q[state][action] = returns_sum[sap] / returns_count[sap]
                policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
            
    
    return Q, policy

In [ ]:
random_policy = create_random_policy(env.action_space.n)
Q, policy = mc_control_importance_sampling(env, num_episodes=500000, behavior_policy=random_policy)

In [ ]:
# For plotting: Create value function from action-value function
# by picking the best action at each state
V = defaultdict(float)
for state, action_values in Q.items():
    action_value = np.max(action_values)
    V[state] = action_value
plotting.plot_value_function(V, title="Optimal Value Function")